In [2]:
#import necessary Libraries
%matplotlib inline
import os
import numpy as np
import random
import tensorflow as tf
import cv2
import keras
from keras.utils import np_utils
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D,BatchNormalization, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from skimage import io
import os
import scipy.misc
from keras.preprocessing import image
from imageio import imread
from keras import regularizers
import csv
from keras.models import Sequential

In [3]:

datasets_path = '/content/drive/MyDrive/Dataset/101_ObjectCategories' #Add the path to the dataset

def load_images(path,n=0): 
    X = [] # intializae array
    Y=[]
    i=-1
    labels = []
    for label in os.listdir(path): #take folder name as a lable of imgaes froo given path's list of directory
        back_path = os.path.join(path,label)
        labels.append(label)  # stores lable in lables named array
        i = i+1
        for filename in os.listdir(back_path):  # for loop to take images from each folder from dataset
            image_path = os.path.join(back_path,filename)  # image path
            img = image.load_img(image_path,target_size=(128,128)) # using keras load_img function load image with size of 128*128
            img = image.img_to_array(img) # convert image to array using keras img_to_array function it will give array 128*128*3
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            
            Y.append(i) # append lable count  in Y array
            X.append(img) # appand converted image to araray in X array
            
    return X,Y,labels

x_train,y,label_data = load_images(datasets_path) # call load_image function and store return value 
X = np.array(x_train) # convert into numpy array

Y = np.array(y)
print(X.shape)

FileNotFoundError: ignored

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25) # split dataset into train and test using train_test_split function

In [ ]:
number_of_classes = 102 # number of classes in dataset
Y_train = np_utils.to_categorical(y_train)
Y_test = np_utils.to_categorical(y_test)

In [ ]:
model = Sequential()
# 1st layer..........
model.add(Conv2D(96, kernel_size = (5, 5), strides = (2,2), activation='relu', padding = 'valid' , data_format='channels_last', input_shape=(128, 128, 3),name = "first"))
model.add(MaxPooling2D(pool_size=(3,3) , strides = (2,2) , padding = 'valid' )) 
model.add(BatchNormalization())

# 2nd layer................
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', strides=(1,1), padding = 'valid', name = "second"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#3rd layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "third"))
model.add(BatchNormalization())

#4th layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "fourth"))
model.add(BatchNormalization())

#5th layer.................
model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation = 'relu', padding='valid' , name = "fifth"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))

# Fully Connected Layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(102, activation = 'softmax')) #As number of nodes in last layer in softmax is number of classes, where each node is probability of classes

In [ ]:
# Compile the model
optimizer = SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, Y_train, batch_size=5, epochs=50, verbose=1, shuffle=True,validation_data=(X_test,Y_test))

In [ ]:
print(model.get_layer("second").get_weights()[0].shape)

In [ ]:
import numpy as np
weights_dic = np.load('/content/drive/MyDrive/Dataset/bvlc_alexnet.npy', encoding='bytes',allow_pickle=True).item()

In [ ]:
model.get_layer("first").set_weights([weights_dic["conv1"][0], weights_dic["conv1"][1]])
# model.get_layer("second").set_weights([weights_dic["conv2"][0], weights_dic["conv2"][1]])
# model.get_layer("third").set_weights([weights_dic["conv3"][0], weights_dic["conv3"][1]])
# model.get_layer("fourth").set_weights([weights_dic["conv4"][0], weights_dic["conv4"][1]])
# model.get_layer("fifth").set_weights([weights_dic["conv5"][0], weights_dic["conv5"][1]])

In [ ]:
from time import time
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_1 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 25, verbose = 1,callbacks=[tensorboard])

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_2 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 10, verbose = 1,callbacks=[tensorboard], shuffle = True)

In [ ]:
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

iter_3 = model.fit(x_train,y_train,validation_data = (x_test, y_test), epochs = 10, verbose = 1,callbacks=[tensorboard], shuffle = True)

In [ ]:
y_pred = model.predict(x_test, y_test)

In [ ]:
print("Testing Accuracy: ",accuracy_score(y_test, y_pred) * 100, "%")